# Download Sen1Floods11 Dataset (Small Subset)

This notebook downloads ~1-2 GB of flood detection data for training.

**Data Source**: [Sen1Floods11](https://github.com/cloudtostreet/Sen1Floods11) - Sentinel-1 flood dataset


## Step 1: Install Required Packages


In [ ]:
# Install packages
!pip install -q earthengine-api geemap geopandas pandas

print("✅ Packages installed")


: 

## Step 2: Setup Google Cloud Authentication


In [ ]:
from google.colab import auth
auth.authenticate_user()

print("✅ Authentication complete")
print("\nNow run the download cells below...")


## Step 3: Create Directories and Download Metadata


In [ ]:
# Create directory structure
!mkdir -p /content/flood_data/S1
!mkdir -p /content/flood_data/S2
!mkdir -p /content/flood_data/labels
!mkdir -p /content/flood_data/metadata

print("✅ Directories created")

# Download CSV metadata files
!gsutil -m cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_train_data.csv /content/flood_data/metadata/
!gsutil -m cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_valid_data.csv /content/flood_data/metadata/
!gsutil -m cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_test_data.csv /content/flood_data/metadata/

print("✅ Metadata files downloaded")


## Step 4: Read Metadata and Select Small Subset (15 samples)


In [ ]:
import pandas as pd
import os

# Read the training data CSV
train_df = pd.read_csv('/content/flood_data/metadata/flood_train_data.csv')

print(f"📊 Total samples available: {len(train_df)}")
print(f"Columns: {list(train_df.columns)}")
print("\nFirst few rows:")
print(train_df.head())

# Select first 15 samples for small dataset (~1-2 GB)
subset_df = train_df.head(15)

print(f"\n✅ Selected {len(subset_df)} samples for download")
print("\nFiles to download:")
for idx, row in subset_df.iterrows():
    print(f"  {idx+1}. {row.iloc[0] if len(row) > 0 else 'N/A'}")


## Step 5: Download Selected S1 Images and Labels


In [ ]:
print(f"📥 Downloading {len(subset_df)} flood events...")
print("This may take 10-15 minutes...\n")

# Download S1 images
print("Downloading S1 SAR images...")
!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand /content/flood_data/S1

print("\nDownloading flood labels...")
!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/LabelHand /content/flood_data/labels

print("\n✅ Download complete!")
!du -sh /content/flood_data


## Step 6: Package and Download to Your Computer


In [ ]:
# Package the dataset
!zip -r /content/flood_dataset.zip /content/flood_data/

# Check size
print("📦 Dataset size:")
!du -sh /content/flood_dataset.zip
print("\n📁 Contents:")
!ls -lh /content/flood_data/S1 | head -5
!ls -lh /content/flood_data/labels | head -5


In [ ]:
# Download the ZIP file to your computer
from google.colab import files

files.download('/content/flood_dataset.zip')

print("✅ Download started!")
print("\nAfter downloading:")
print("1. Extract flood_dataset.zip")
print("2. Copy S1 images to: C:\\Users\\Nishant Raj\\Desktop\\Flood detection\\data\\raw\\S1\\")
print("3. Copy labels to: C:\\Users\\Nishant Raj\\Desktop\\Flood detection\\data\\raw\\labels\\")
print("4. Run: python src/training/train.py --model resnet")
